In [34]:
import os
import pandas as pd
from datasets import load_dataset
import random
import warnings
from datasets import Dataset, DatasetDict

warnings.simplefilter(action='ignore', category=FutureWarning)



Data Path

In [26]:
data_dir = os.path.join('dataset')
arguments_training_filepath = os.path.join(data_dir, 'arguments-training.tsv')
arguments_validation_filepath = os.path.join(data_dir, 'arguments-validation.tsv')
arguments_validation_filepath_zhihu = os.path.join(data_dir, 'arguments-validation-zhihu.tsv')
arguments_test_filepath = os.path.join(data_dir, 'arguments-test.tsv')

labels_training_filepath = os.path.join(data_dir, 'labels-training.tsv')
labels_validation_filepath = os.path.join(data_dir, 'labels-validation.tsv')
labels_validation_filepath_zhihu = os.path.join(data_dir, 'labels-validation-zhihu.tsv')
labels_test_filepath = os.path.join(data_dir, 'labels-test.tsv')


In [27]:
HIGHER_ORDER_VALUES = ['Opennes to change', 'Self-transcendence', 'Conservation', 'Self-enhancement']

HIGHER_ORDER_VALUES_AND_SUB = {'Opennes to change':['Self-direction: thought', 'Self-direction: action', 'Stimulation', 'Hedonism'], 
                       'Self-transcendence':['Humility', 'Benevolence: caring','Benevolence: dependability', 'Universalism: concern', 'Universalism: nature', 'Universalism: tolerance', 'Universalism: objectivity'], 
                       'Conservation':['Humility', 'Tradition','Conformity: interpresonal','Conformity: rules', 'Security: societal', 'Security: personal', 'Face'], 
                       'Self-enhancement':['Face', 'Power: dominance', 'Power: resources', 'Achievement', 'Hedonism']}


LABELS = ['Self-direction thought', 'Self-direction action', 'Stimulation', 'Hedonism', 'Achievement', 'Power dominance', 'Power resources', 'Face', 'Security personal', 'Security societal', 'Tradition', 'Conformity rules', 'Conformity interpersonal', 'Humility', 'Benevolence caring', 'Benevolence dependability', 'Universalism concern', 'Universalism nature', 'Universalism tolerance', 'Universalism objectivity']
PROMPT_FORMATS = ["The premise: '{}' is '{}'. The conclusion is '{}'\n. Question: Which value category does the argument belong to? Options: {} \n",
                  "Premise: {}\nStance: {}\nConclusion: {}. Value category: {}\n Question: Which value category does the argument belong to?\n",
                  "Argument: {}. {}. {}. Value category: {}\n Question: Which value category does the argument belong to?\n"]

ENSEMBLE_PROMPT = ["The premise '{}' is '{}'. The conclusion is '{}'. Which of the following higher order values does that support, there can be more than one option? Options: {}\n",
                   "The premise '{}' is '{}'. The conclusion is '{}'. This falls in the higher order value of '{}'. Which of the following value categories does that support? {}\n"]


def convert_binary_labels_to_string(df):
    label_names = df.columns[1:]
    labels = []

    for index, row in df.iterrows():
        binary_values = row.values[1:]
        string_labels = ''
        for i, value in enumerate(binary_values):
            if value == 1:
                string_labels += label_names[i] + ', '
        labels.append(string_labels[:-2])
    return labels

def ensemble_prompt_higher_order(df):
    """Creates an ensemble prompt for each argument with the first prompt format"""
    
    template = ENSEMBLE_PROMPT[0]
    prompts = [
                template.format(row['Premise'], row['Stance'], row['Conclusion'], ', '.join(HIGHER_ORDER_VALUES))
                for _, row in df.iterrows()
    ]
    df['ensemble'] = prompts
    return df


def single_shot_prompt(df):
    """Creates a single shot prompt for each argument with the first prompt format"""
    
    template = PROMPT_FORMATS[0] # use the first template 
    prompts = [
                template.format(row['Premise'], row['Stance'], row['Conclusion'], ', '.join(LABELS))
                for _, row in df.iterrows()
    ]
    df['single_shot_prompt'] = prompts
    return df

def few_shot_prompt(df, num_shots=1, prompt_format=0, random_seed=46):
    """Creates a few shot prompt for each argument"""

    prompt_format = PROMPT_FORMATS[prompt_format]
    
    selected_arguments = df.sample(n=num_shots, random_state=random_seed)
    few_shot_prompts = [
        prompt_format.format(row['Premise'], row['Stance'], row['Conclusion'], ', '.join(LABELS)) + f"Answer: {random.choice(LABELS)}\n"
        prompt_format.format(row['Premise'], row['Stance'], row['Conclusion'], ', '.join(LABELS)) + f"Answer: {', '.join(random.sample(LABELS, 2))}\n"
        for _, row in selected_arguments.iterrows()
    ]
    df['few_shot_prompt'] = df.apply(lambda row: ''.join(few_shot_prompts) + prompt_format.format(row['Premise'], row['Stance'], row['Conclusion'], ', '.join(LABELS)) + f"Answer: \n", axis=1)
    return df


# used for testing different prompt formats 
def prompt_formats(df):
    prompts = [
        [
            prompt.format(row['Premise'], row['Stance'], row['Conclusion'], ', '.join(LABELS))
            for prompt in PROMPT_FORMATS
        ]
        for _, row in df.iterrows()
    ]

    df['prompt_formats'] = prompts
    return df

def labels_to_multi_choice():
    """Converts the labels to a multi choice format"""
    multi_choice_format = "{}: {}"
    multi_choice_options = []

    for index, label in enumerate(LABELS):
        multi_choice_option = multi_choice_format.format(chr(65 + index), label)
        multi_choice_options.append(multi_choice_option)

    return multi_choice_options

def label_to_vector(df):
    """Converts the labels to a vector"""
    label_names = df.iloc[:, 1:]
    return label_names.values.tolist()

Load the data

In [28]:
train = pd.read_csv(arguments_training_filepath, encoding='utf-8', sep='\t', header=0)
validation = pd.read_csv(arguments_validation_filepath, encoding='utf-8', sep='\t', header=0)
validation_zhihu = pd.read_csv(arguments_validation_filepath_zhihu, encoding='utf-8', sep='\t', header=0)
test = pd.read_csv(arguments_test_filepath, encoding='utf-8', sep='\t', header=0)

labels_training = pd.read_csv(labels_training_filepath, encoding='utf-8', sep='\t', header=0)
labels_validation = pd.read_csv(labels_validation_filepath, encoding='utf-8', sep='\t', header=0)
labels_validation_zhihu = pd.read_csv(labels_validation_filepath_zhihu, encoding='utf-8', sep='\t', header=0)
labels_test = pd.read_csv(labels_test_filepath, encoding='utf-8', sep='\t', header=0)


## add vector labels to the dataframes 

In [29]:
train['label_vector'] = label_to_vector(labels_training)
validation['label_vector'] = label_to_vector(labels_validation)
validation_zhihu['label_vector'] = label_to_vector(labels_validation_zhihu)
test['label_vector'] = label_to_vector(labels_test)
train.head()

,Argument ID,Conclusion,Stance,Premise,label_vector
0,A01001,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
1,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
2,A01003,We should abandon marriage,against,marriage is the ultimate commitment to someone...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,A01004,We should ban naturopathy,against,it provides a useful income for some people,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


## Combine the data and labels into a single dataframe

In [30]:
train['label_string'] = convert_binary_labels_to_string(labels_training)
validation['label_string'] = convert_binary_labels_to_string(labels_validation)
validation_zhihu['label_string'] = convert_binary_labels_to_string(labels_validation_zhihu)
test['label_string'] = convert_binary_labels_to_string(labels_test)

# train.iloc[7]['label_string']
train.iloc[:20]

,Argument ID,Conclusion,Stance,Premise,label_vector,label_string
0,A01001,Entrapment should be legalized,in favor of,if entrapment can serve to more easily capture...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",Security: societal
1,A01002,We should ban human cloning,in favor of,we should ban human cloning as it will only ca...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...",Security: societal
2,A01003,We should abandon marriage,against,marriage is the ultimate commitment to someone...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Self-direction: action
3,A01004,We should ban naturopathy,against,it provides a useful income for some people,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",Security: personal
4,A01005,We should ban fast food,in favor of,fast food should be banned because it is reall...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...",Security: personal
5,A01006,We should end the use of economic sanctions,against,sometimes economic sanctions are the only thin...,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","Power: dominance, Security: societal"
6,A01007,We should abolish capital punishment,against,capital punishment is sometimes the only optio...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, ...","Security: societal, Conformity: rules, Univers..."
7,A01008,We should ban factory farming,against,factory farming allows for the production of c...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","Security: personal, Benevolence: caring, Unive..."
8,A01009,We should fight for the abolition of nuclear w...,against,nuclear weapons help keep the peace in uncerta...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","Security: societal, Universalism: concern"
9,A01010,We should prohibit school prayer,against,it should be allowed if the student wants to p...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","Self-direction: thought, Self-direction: actio..."


## Add higher level labels to dataframe

In [18]:
higher_order_values_df = pd.DataFrame(columns=['higher_order_value', 'value_category'])
for higher_order_value in HIGHER_ORDER_VALUES_AND_SUB:
    for value_category in HIGHER_ORDER_VALUES_AND_SUB[higher_order_value]:
        higher_order_values_df = higher_order_values_df.append({'higher_order_value': higher_order_value, 'value_category': value_category}, ignore_index=True)

# higher_order_values_df

In [19]:
# # add the higher order values to the labels of the training, validation and test set
# def add_higher_order_values(df):
#     df['higher_order_value'] = df['label_string'].apply(lambda x: higher_order_values_df[higher_order_values_df['value_category'].isin(x)]['higher_order_value'].unique())
#     return df

# train = add_higher_order_values(train)
# validation = add_higher_order_values(validation)
# validation_zhihu = add_higher_order_values(validation_zhihu)
# test = add_higher_order_values(test)

# train.head()

## Add promts to dataframe

In [37]:
train = single_shot_prompt(train)
train = few_shot_prompt(train, num_shots=1, prompt_format=0, random_seed=46)
train = ensemble_prompt_higher_order(train) # 

train.iloc[0]['few_shot_prompt']

"The premise: 'people should keep their religion to themselves' is 'in favor of'. The conclusion is 'We should ban missionary work'\n. Question: Which value category does the argument belong to? Options: Self-direction thought, Self-direction action, Stimulation, Hedonism, Achievement, Power dominance, Power resources, Face, Security personal, Security societal, Tradition, Conformity rules, Conformity interpersonal, Humility, Benevolence caring, Benevolence dependability, Universalism concern, Universalism nature, Universalism tolerance, Universalism objectivity \nAnswer: Power resources, Conformity interpersonal\nThe premise: 'if entrapment can serve to more easily capture wanted criminals, then why shouldn't it be legal?' is 'in favor of'. The conclusion is 'Entrapment should be legalized'\n. Question: Which value category does the argument belong to? Options: Self-direction thought, Self-direction action, Stimulation, Hedonism, Achievement, Power dominance, Power resources, Face, Se

## Setup the model

In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")

2023-05-30 14:42:10.321576: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Select 10 random samples 

In [22]:
train_10 = train.sample(10)
train_10

,Argument ID,Conclusion,Stance,Premise,label_vector,label_string,single_shot_prompt,few_shot_prompt,ensemble
33,A02014,We should abolish the right to keep and bear arms,against,we should not abolish the right to keep and be...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, ...","Self-direction: action, Conformity: rules, Ben...",The premise: 'we should not abolish the right ...,The premise: 'people should keep their religio...,The premise 'we should not abolish the right t...
2531,A21481,We should abolish safe spaces,against,Safe spaces are necessary for many people to h...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","Security: personal, Benevolence: caring, Unive...",The premise: 'Safe spaces are necessary for ma...,The premise: 'people should keep their religio...,The premise 'Safe spaces are necessary for man...
88,A05009,We should ban naturopathy,in favor of,naturopathy may have no scientific evidence.,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...","Security: personal, Security: societal, Univer...",The premise: 'naturopathy may have no scientif...,The premise: 'people should keep their religio...,The premise 'naturopathy may have no scientifi...
2569,A22025,We should subsidize journalism,in favor of,subsidizing journalism allows more voices to b...,"[1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, ...","Self-direction: thought, Power: dominance, Fac...",The premise: 'subsidizing journalism allows mo...,The premise: 'people should keep their religio...,The premise 'subsidizing journalism allows mor...
4622,A18084,We should ban missionary work,against,independent religious organisations should be ...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","Self-direction: thought, Self-direction: actio...",The premise: 'independent religious organisati...,The premise: 'people should keep their religio...,The premise 'independent religious organisatio...
3414,A24021,We should ban cosmetic surgery for minors,in favor of,cosmetic surgery for minors does not address s...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ...","Security: personal, Humility, Universalism: ob...",The premise: 'cosmetic surgery for minors does...,The premise: 'people should keep their religio...,The premise 'cosmetic surgery for minors does ...
4350,A20107,We should adopt atheism,against,adopting atheism would violate freedom of reli...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","Self-direction: thought, Self-direction: actio...",The premise: 'adopting atheism would violate f...,The premise: 'people should keep their religio...,The premise 'adopting atheism would violate fr...
977,A18124,We should stop the development of autonomous cars,against,autonomous cars can give many people the freed...,"[0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, ...","Self-direction: action, Stimulation, Hedonism,...",The premise: 'autonomous cars can give many pe...,The premise: 'people should keep their religio...,The premise 'autonomous cars can give many peo...
1895,A20214,We should end racial profiling,in favor of,there is no valid argument in favor of racial ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Universalism: concern, Universalism: objectivity",The premise: 'there is no valid argument in fa...,The premise: 'people should keep their religio...,The premise 'there is no valid argument in fav...
494,A12129,We should legalize prostitution,against,"prostitution is linked to many rapes, molestat...","[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...","Security: personal, Security: societal, Univer...",The premise: 'prostitution is linked to many r...,The premise: 'people should keep their religio...,The premise 'prostitution is linked to many ra...


## Zero-shot

In [23]:
def query_from_list(query):
    inputs = tokenizer(query, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=20)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

for index, row in train_10.iterrows():
    result = query_from_list(row['single_shot_prompt'])
    print(f"Prompt: {row['single_shot_prompt']} \n Prediction: {result[0]}\n True Label: {row['label_string']}\n")
    print(60*'-')

Prompt: The premise: 'we should not abolish the right to keep and bear arms as everyone has a right to keep guns as a mean of protection as stated in the constitution.' is 'against'. The conclusion is 'We should abolish the right to keep and bear arms'
. Question: Which value category does the argument belong to? Options: Self-direction thought, Self-direction action, Stimulation, Hedonism, Achievement, Power dominance, Power resources, Face, Security personal, Security societal, Tradition, Conformity rules, Conformity interpersonal, Humility, Benevolence caring, Benevolence dependability, Universalism concern, Universalism nature, Universalism tolerance, Universalism objectivity 
 
 Prediction: Universalism nature
 True Label: Self-direction: action, Conformity: rules, Benevolence: caring

------------------------------------------------------------
Prompt: The premise: 'Safe spaces are necessary for many people to have strong mental and emotional health, and we should advocate to pro

## Few-shot

In [24]:
for index, row in train_10.iterrows():
    result = query_from_list(row['few_shot_prompt'])
    print(f"{row['few_shot_prompt']:<24} {result[0]}\n True Label: {row['label_string']}\n")
    # break

The premise: 'people should keep their religion to themselves' is 'in favor of'. The conclusion is 'We should ban missionary work'
. Question: Which value category does the argument belong to? Options: Self-direction thought, Self-direction action, Stimulation, Hedonism, Achievement, Power dominance, Power resources, Face, Security personal, Security societal, Tradition, Conformity rules, Conformity interpersonal, Humility, Benevolence caring, Benevolence dependability, Universalism concern, Universalism nature, Universalism tolerance, Universalism objectivity 
Answer: Face, Conformity interpersonal
The premise: 'we should not abolish the right to keep and bear arms as everyone has a right to keep guns as a mean of protection as stated in the constitution.' is 'against'. The conclusion is 'We should abolish the right to keep and bear arms'
. Question: Which value category does the argument belong to? Options: Self-direction thought, Self-direction action, Stimulation, Hedonism, Achieve

## Prompt ensemble for higher order values

In [25]:
for index, row in train_10.iterrows():
    result = query_from_list(row['ensemble'])
    print(f"Prompt1:\t{row['ensemble']} Prediction higher_order_value:\t {result[0]}\n True higher_order_value:\t{row['higher_order_value']}")
    # template = "The premise '{}' is '{}'. The conclusion is '{}'. This falls in the higher order value of '{}'. Which of the following value categories does that support? {}\n"
    template = ENSEMBLE_PROMPT[1]
    values = HIGHER_ORDER_VALUES_AND_SUB[result[0]]
    prompt = template.format(row['Premise'], row['Stance'], row['Conclusion'], result[0], values)
    result = query_from_list(prompt)
    print(f"Prompt2:\t {prompt}Predicted value category:\t {result[0]}\n True value category:\t {row['label_string']}\n")
    print(10*'-------------------')

KeyError: 'higher_order_value'

In [ ]:
from datasets import load_dataset, Dataset, load_from_disk
dataset_path = '../datasets/touche23_prompt'
dataset = load_from_disk(dataset_path)
dataset['train'][7]
